 ## Prise en main de la librairie GEDCOM

### Conventions : 
Tag for Spouse FamiliyElement : 'FAMS'
$\\$
Tag for Childrens FamiliyElement : 'FAMC'

In [11]:
from gedcom.element.individual import IndividualElement
from gedcom.element.element import Element
from gedcom.parser import Parser
from gedcom.element.family import FamilyElement
#from gedcom.element.element import Element
#from gedcom.element.file import FileElement
#from gedcom.element.object import ObjectElement

In [12]:
file_path = 'Queen_Eliz_II.ged'
gedcom_parser = Parser()
gedcom_parser.parse_file(file_path)
root_child_elements = gedcom_parser.get_root_child_elements()

def find_child_data(first_names,last_name):
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                if element.given_name_match(first_names):
                    L1 = []
                    L2 = []
                    (first, last) = element.get_name()
                    print("Information about " + first + " " + last + ":")
                    data = element.get_child_elements()
                    for child in data :
                        L1 += [child]
                        L2 += [child.get_tag()]
                    return L1, L2
    return "No matching person"

In [13]:
find_child_data("Elizabeth II Alexandra Mary", "Windsor")

Information about Elizabeth II Alexandra Mary Windsor:


([<gedcom.element.element.Element at 0x2670898f2b0>,
 ['RIN', '_UID', '_UPD', 'NAME', 'SEX', 'BIRT', 'FAMS', 'FAMC'])

In [14]:
def find_IndividualElement(first_names,last_name):
    """
    Get Individual element of ged with first names and last name

    Parameters
    ---
    first_names : str
        complete first names of the individual
    last_name : str
        last names of the individual 

    Returns 
    ---
    bool 
        Tells if the individual was found
    IndividualElement
        IndividualElement corresponding to the individual in the ged

    """
    for element in root_child_elements:
        if isinstance(element, IndividualElement):
            if element.surname_match(last_name):
                return (True, element)
    return [False, element]

In [15]:
def get_Spouse_FamilyElement(Individual):
    """
    Get Spouse FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMS")
    

In [16]:
def get_Children_FamilyElement(Individual):
    """
    Get Childrens FamilyElements of an individual

    Parameters
    ---
    Individual : IndividualElement

    Returns
    ---
    list of FamilyElement 
    """
    return gedcom_parser.get_families(Individual,"FAMC")

In [17]:
get_Spouse_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [18]:
get_Children_FamilyElement(find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1])

In [19]:
A = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
B = find_IndividualElement("George VI", "Windsor")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [20]:
C = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
D = find_IndividualElement("George V", "Windsor")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [21]:
E = find_IndividualElement("Elizabeth II Alexandra Mary", "Windsor")[1]
F = find_IndividualElement("Edward VII", "Wettin")[1]
gedcom_parser.find_path_to_ancestor(A,B,None)[0].get_name()

('George V', 'Windsor')

In [78]:
def get_spouse(Individual):
    Family = get_Spouse_FamilyElement(Individual)[0]
    return gedcom_parser.get_family_members(Family,"FAMILY_MEMBERS_TYPE_ALL")

In [88]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])

In [85]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[0].get_name()

('George VI', 'Windsor')

In [86]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[1].get_name()

('Mary of Teck (May)', '')

In [87]:
get_spouse(find_IndividualElement("George VI", "Windsor")[1])[2].get_name()

('George VI', 'Windsor')

In [89]:
get_spouse(find_IndividualElement('Mary of Teck (May)', '')[1])[0].get_name()

('SIR WILLIAM VII KNIGHT ENGLAND', 'Gascoigne')